In [2]:
import os

In [1]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "brain_tumor_dataset")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [15]:
import time

In [16]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [17]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [18]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [19]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Reading yaml file from: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/config/config.yaml
[2023-07-06 23:26:14,597: INFO: common: yaml file: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/config/config.yaml loaded successfully]
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Reading yaml file from: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/params.yaml
[2023-07-06 23:26:14,600: INFO: common: yaml file: /home/rhythm/Desktop/Job/prep/Projects/brain_tumor_classification/brain-tumor-classification/params.yaml loaded successfully]
[2023-07-06 23:26:14,601: INFO: common: created directory at: artifacts]
[2023-07-06 23:26:14,601: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoin

2023-07-06 23:26:14.602849: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-07-06 23:26:14.602880: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2023-07-06 23:26:14.602909: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


Found 50 images belonging to 2 classes.
Found 203 images belonging to 2 classes.


2023-07-06 23:26:15.156411: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-07-06 23:26:15.157722: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2995200000 Hz


 1/12 [=>............................] - ETA: 45s - loss: 0.8200 - accuracy: 0.5625

2023-07-06 23:26:19.378161: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2023-07-06 23:26:19.381118: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.


 2/12 [====>.........................] - ETA: 33s - loss: 8.3315 - accuracy: 0.4375

2023-07-06 23:26:22.699574: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2023-07-06 23:26:22.703253: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2023-07-06 23:26:22.708090: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at_2023-07-06-23-26-14/train/plugins/profile/2023_07_06_23_26_22
2023-07-06 23:26:22.709296: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at_2023-07-06-23-26-14/train/plugins/profile/2023_07_06_23_26_22/rhythm.trace.json.gz
2023-07-06 23:26:22.715919: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at_2023-07-06-23-26-14/train/plugins/profile/2023_07_06_23_26_22
2023-07-06 23:26:22.716503: I tensorflow

12/12 [==============================] - 48s 4s/step - loss: 15.0258 - accuracy: 0.5294 - val_loss: 27.0437 - val_accuracy: 0.6042
